In [1]:
import sys
import os
from typing import Tuple
import pandas as pd 
import mlflow as mf

## replace with your specific paths
sys.path.append("/home/aamir07/mlops2_with_dagster/") 
ARTIFACT_PATH = "/home/aamir07/mlops2_with_dagster/artifacts/mlflow_artfacts" 
LOGGER_FOLDER_PATH = "/home/aamir07/mlops2_with_dagster/notebooks"

from mlops2_with_dagster import model_pipeline, features_pipeline
from pathlib import Path
from joblib import dump, load
from hamilton import driver, base
from mlops2_with_dagster.utils import get_project_dir, printse
from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)

from ortho import BaseKernel
from ortho.utils import Logger
from ortho.ortho.decorators import task
from ortho.ortho.callbacks import MlFlowCallBack, LoggerCallBack


logger = Logger().logger()

   

        
        

None


2023-09-15 00:31:24.278210: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 00:31:24.376784: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 00:31:24.378965: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 00:31:27,441 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-09-15 00:31:27.740307: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-15 00:31:27,839 - DEBUG - Creating converter from 7 to 5
2023-09-15 00:31:27,841 - DEBUG - Creating converter from 5 to 7
2

In [2]:
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/aamir07/mlops2_with_dagster


In [3]:
train_data : Path = project_dir/"data/train.csv"
test_data : Path = project_dir/"data/test.csv"

In [4]:
train_features_file: str = f"{project_dir}/warehouse/transformed_data.parquet"
target_file: str = f"{project_dir}/warehouse/target.parquet"

In [5]:
# parameters
train_features = pd.read_parquet(train_features_file)
target = pd.read_parquet(target_file)

In [6]:
class Train(BaseKernel):
    
    def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
        super().__init__(callbacks,
                       experiment_name,
                       run_name,
                       load_from_artifact)
        
        self.index_col = 'passengerid'
        self.target_col = "survived"
        self.cat_cols = ["sex", "cabin", "embarked"]
        self.config = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'categorical_columns': self.cat_cols
        }
        
        self.config_model = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'random_state': 42,
            'max_depth': None,
            'validation_size_fraction': 0.33,
            't': 0.5
        }
    
    @task(build_on_previous_run=True, end_mlflow_run=False)       
    def fit(self, transformed_data=None, target=None):
        training_adapter = base.SimplePythonGraphAdapter(base.DictResult())
        training_dr = driver.Driver(self.config_model, 
                           model_pipeline,
                           adapter=training_adapter)
        dtraining = dict(
            final_feature_matrix = transformed_data,
            target = target.target
        )
        output_nodes = ['fit_clf', 'train_predictions', 'valid_predictions']
        
        output = training_dr.execute(output_nodes, inputs = dtraining)
        
        data = {"training_outputs": output }
        payload = {"artifact_path": ARTIFACT_PATH,
                "params":self.config_model}
        return data, payload
        
    def run(self, *args, **kwargs):
        return self.fit(*args, **kwargs)
        
        
        
        
    

In [7]:
import dagstermill
import pickle
trainer = Train(experiment_name="Mlflow_with_Dagster",
                          run_name="BadamBhum",
                          callbacks = [LoggerCallBack(log_folder=LOGGER_FOLDER_PATH,
                                                      kernel_names=["fit"]),
                                       MlFlowCallBack(kernel_names=["fit"])],
                         )

# mf.end_run() #Document why it his here 
output = trainer.run(transformed_data=train_features, target=target)[0]["training_outputs"]
dagstermill.yield_result(output, output_name="training_outputs")

2023-09-15 00:31:30,872 - DEBUG - Starting new HTTP connection (1): 127.0.0.1:5002
2023-09-15 00:31:31,339 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=Mlflow_with_Dagster HTTP/1.1" 200 241
2023-09-15 00:31:31,349 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 00:31:31,390 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=a05f805113ff42a3b54371547a26d49b&run_id=a05f805113ff42a3b54371547a26d49b HTTP/1.1" 200 1572
2023-09-15 00:31:31,399 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 00:31:31,500 - DEBUG - http://127.0.0.1:5002 "POST /api/2.0/mlflow/runs/update HTTP/1.1" 200 423
2023-09-15 00:31:31,508 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 00:31:31,561 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=a05f805113ff42a3b54371547a26d49b&run_id=a05f805113ff42a3b54371547a26d49b HTTP/1.1" 200 1571
2023-09-15 00:31:31,566 - INFO - Successfuly initiated run

{'fit_clf': RandomForestClassifier(random_state=42),
 'train_predictions': (array([0.66   , 0.07   , 0.14   , ..., 0.1    , 0.50875, 0.22   ]),
  array([1, 0, 0, ..., 0, 1, 0])),
 'valid_predictions': (array([0.47      , 0.19      , 0.87      , ..., 0.31533333, 0.32      ,
         0.42      ]),
  array([0, 0, 1, ..., 0, 0, 0]))}